In [1]:
!nvidia-smi

Wed Jul  7 16:54:50 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   57C    P8     6W /  N/A |   1344MiB /  6144MiB |    ERR!      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch
import os
import pandas as pd

import sys
sys.path.append('..')

from deepmatcher.data.custom_dataset import DeepMatcherDataset
from deepmatcher.data.encoder.text_encoders import HFTextEncoder
from deepmatcher.data.encoder.image_encoders import DINOImageEncoder
from deepmatcher.optim import Optimizer

from torch.utils.data import DataLoader
from deepmatcher.models.core import MatchingModel

In [3]:
dataset_dir = "sample_data/itunes-amazon/"
train_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('train:', train_df.shape)
val_df = pd.read_csv(os.path.join(dataset_dir, 'validation.csv'))
print('val:', val_df.shape)
test_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('test:', test_df.shape)

for df in (train_df, val_df, test_df):
    df['left_image'] = df['right_image'] = os.path.join(dataset_dir, 'test_img.jpg')
train_df.head()

train: (323, 18)
val: (108, 18)
test: (323, 18)


,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released,left_image,right_image
0,448,0,Baby When the Light ( David Guetta & Fred Rist...,David Guetta,Pop Life ( Extended Version ) [ Bonus Version ],"Dance , Music , Rock , Pop , House , Electroni...",$ 1.29,‰ ãÑ 2007 Gum Records,6:17,18-Sep-07,Revolver ( Madonna Vs. David Guetta Feat . Lil...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:18,"August 21 , 2009",sample_data/itunes-amazon/test_img.jpg,sample_data/itunes-amazon/test_img.jpg
1,287,1,Outversion,Mark Ronson,Version,"Pop , Music , R&B / Soul,Soul,Dance,Rock,Jazz,...",$ 0.99,2007 Mark Ronson under exclusive license to SO...,1:50,10-Jul-07,Outversion,Mark Ronson,Version [ Explicit ],Pop,$ 0.99,( c ) 2011 J'adore Records,1:50,"July 10 , 2007",sample_data/itunes-amazon/test_img.jpg,sample_data/itunes-amazon/test_img.jpg
2,534,0,Peer Pressure ( feat . Traci Nelson ),Snoop Dogg,Doggumentary,"Hip-Hop/Rap , Music , Rock , Gangsta Rap , Wes...",$ 1.29,"‰ ãÑ 2011 Capitol Records , LLC . All rights r...",4:07,29-Mar-11,Boom ( ( Feat . T-Pain ) [ Edited ] ),Snoop Dogg,Doggumentary [ Edited ],"Rap & Hip-Hop , West Coast",$ 1.29,"( C ) 2011 Capitol Records , LLC",3:50,"March 29 , 2011",sample_data/itunes-amazon/test_img.jpg,sample_data/itunes-amazon/test_img.jpg
3,181,1,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Stars Come Out ( Dillon Francis Remix ),Zedd,Stars Come Out [ Dillon Francis Remix ],Dance & Electronic,$ 1.29,2012 Dim Mak Inc.,4:08,"May 20 , 2014",sample_data/itunes-amazon/test_img.jpg,sample_data/itunes-amazon/test_img.jpg
4,485,0,Jump ( feat . Nelly Furtado ),Flo Rida,R.O.O.T.S. ( Deluxe Version ),"Hip-Hop/Rap , Music",$ 1.29,‰ ãÑ 2009 Atlantic Recording Corporation for t...,3:28,30-Mar-09,"Yayo [ Feat . Brisco , Billy Blue , Ball Greez...",Flo Rida,R.O.O.T.S. ( Route Of Overcoming The Struggle ...,Rap & Hip-Hop,$ 1.29,"( C ) 2012 Motown Records , a Division of UMG ...",7:53,"March 30 , 2009",sample_data/itunes-amazon/test_img.jpg,sample_data/itunes-amazon/test_img.jpg


In [4]:
text_encoder = HFTextEncoder('albert-base-v2', trainable=False, seqtovec='mean')
image_encoder = DINOImageEncoder(device='cuda')


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.decoder.weight', 'predictions.bias', 'predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.bias', 'predictions.LayerNorm.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
label_column = 'label'
text_columns = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_col = 'image'
tokenizer = text_encoder.tokenizer

def get_dataloader(data_df, label_column, text_cols, imaeg_col, tokenizer, batch_size, shuffle, num_workers):
    dataset = DeepMatcherDataset(
        data_df=data_df,
        label_col=label_column,
        text_cols=text_columns,
        image_col=image_col,
        tokenizer=tokenizer
        )
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=dataset.collate_fn)

batch_size = 16
num_workers = 4

train_dataloader = get_dataloader(train_df, label_column, text_columns, image_col, tokenizer, batch_size, True, num_workers)
val_dataloader = get_dataloader(val_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)
test_dataloader = get_dataloader(test_df, label_column, text_columns, image_col, tokenizer, batch_size, False, num_workers)

received data_df : (323, 20)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : image
supplied label column : label
received data_df : (108, 20)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : image
supplied label column : label
received data_df : (323, 20)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : image
supplied label column : label


In [6]:
model = MatchingModel(
    text_encoder=text_encoder,
    image_encoder=image_encoder,
    attr_summarizer=None,
    attr_condense_factor='auto',
    attr_comparator='concat',
    attr_merge='concat',
    classifier='2-layer-highway',
    hidden_size=300,
    text_attrs=text_columns,
    image_attr='image'
    )

text_attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_attr: image
prefixes: ('left_', 'right_')
attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released', 'image']
all attrs: ['left_Song_Name', 'right_Song_Name', 'left_Artist_Name', 'right_Artist_Name', 'left_Album_Name', 'right_Album_Name', 'left_Genre', 'right_Genre', 'left_Price', 'right_Price', 'left_CopyRight', 'right_CopyRight', 'left_Time', 'right_Time', 'left_Released', 'right_Released', 'left_image', 'right_image']


In [7]:
model.initialize(next(iter(train_dataloader))['attrs'])
model

/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/functional.py:3609: UserWarning: Default upsampling behavior when mode=bicubic is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/functional.py:3657: UserWarning: The default behavior for interpolate/upsample with float scale_factor changed in 1.6.0 to align with other frameworks/libraries, and now uses scale_factor directly, instead of relying on the computed output size. If you wish to restore the old behavior, please set recompute_scale_factor=True. See the documentation of nn.Upsample for details. 
  warnings.warn(
/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. T

MatchingModel(
  (text_encoder): HFTextEncoder(
    (model): AlbertModel(
      (embeddings): AlbertEmbeddings(
        (word_embeddings): Embedding(30000, 128, padding_idx=0)
        (position_embeddings): Embedding(512, 128)
        (token_type_embeddings): Embedding(2, 128)
        (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0, inplace=False)
      )
      (encoder): AlbertTransformer(
        (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
        (albert_layer_groups): ModuleList(
          (0): AlbertLayerGroup(
            (albert_layers): ModuleList(
              (0): AlbertLayer(
                (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                (attention): AlbertAttention(
                  (query): Linear(in_features=768, out_features=768, bias=True)
                  (key): Linear(in_features=768, out_features=768, bias=True)
                

In [8]:
optimizer = Optimizer(lr=1e-3, lr_decay=1e-1, start_decay_at=3)

In [9]:
model.run_train(
    train_dataloader,
    val_dataloader,
    best_save_path='best_mode.pth',
    epochs=15,
    )

* Number of trainable parameters: 1413902
===>  TRAIN Epoch 1
/home/sour4bh/.local/lib/python3.8/site-packages/torch/nn/functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0% [████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:21
Finished Epoch 1 || Run Time:   81.1 | Load Time:    1.3 || F1:  16.67 | Prec:  42.11 | Rec:  10.39 || Ex/s:   3.92

===>  EVAL Epoch 1
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
 

KeyboardInterrupt: 

In [ ]:
model.run_eval(test_dataset)